# Weekend Movie Trip

Data from https://github.com/fivethirtyeight/data/tree/master/soccer-spi, using soccer dataset.

In [1]:
import pandas as pd

### Load data

In [5]:
spi_matches = pd.read_csv("../data/spi_matches.csv")
spi_rankings = pd.read_csv("../data/spi_global_rankings.csv")
spi_rankings_intl = pd.read_csv("../data/spi_global_rankings_intl.csv")

In [6]:
spi_matches.head()

,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,0.1157,...,32.4,67.7,0.0,1.0,0.97,0.63,0.43,0.45,0.00,1.05
1,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,0.2617,...,53.7,22.9,2.0,2.0,2.45,0.77,1.75,0.42,2.10,2.10
2,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,0.2921,...,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
3,2016-08-13,2411,Barclays Premier League,Burnley,Swansea City,58.98,59.74,0.4482,0.2663,0.2854,...,36.5,29.1,0.0,1.0,1.24,1.84,1.71,1.56,0.00,1.05
4,2016-08-13,2411,Barclays Premier League,Middlesbrough,Stoke City,56.32,60.35,0.4380,0.2692,0.2927,...,33.9,32.5,1.0,1.0,1.40,0.55,1.13,1.06,1.05,1.05


In [7]:
spi_rankings.head()

,rank,prev_rank,name,league,off,def,spi
0,1,1,Manchester City,Barclays Premier League,2.92,0.20,93.78
1,2,2,Barcelona,Spanish Primera Division,3.12,0.38,92.41
2,3,3,Real Madrid,Spanish Primera Division,2.99,0.38,91.75
3,4,4,Bayern Munich,German Bundesliga,2.94,0.40,90.93
4,5,5,Juventus,Italy Serie A,2.66,0.29,90.72


In [8]:
spi_rankings_intl.head()

,rank,name,confed,off,def,spi
0,1,Brazil,CONMEBOL,3.07,0.27,93.09
1,2,Spain,UEFA,3.53,0.51,92.53
2,3,Belgium,UEFA,3.02,0.55,88.64
3,4,France,UEFA,2.82,0.50,87.71
4,5,Germany,UEFA,2.94,0.57,87.59
